In [2]:
import sys, io
import math
import numpy as np
import pandas as pd
import random
import re
from scipy.optimize import nnls
from fractions import Fraction
from sympy import factorint
from itertools import product
from sympy.ntheory import factorint
from typing import List, Optional, Tuple

# The $2n+1$ easy set

The goal of this notebook is to directly generate all "easy $2n+1$" operations by recognizing the constraints of the mrTup2n_plus1 function identified in the Modulus_C_Sum notebook

## $'01'\ \mathbin{++}\  label \ \mapsto \ \frac{4n-1}{3}$ 

In this notebook, noticed this general pattern.  May come in handy, we can generate many of these since these are the inverses of the Collatz chain process.

Prepending 01 to any integer generates the value $\frac{4n-1}{3}$.  So when:
- $4n \equiv 1 \pmod 3$ we get a new integer 
- $n \equiv 1 \pmod 3$  we get a new integer

This is the obvious inverse of $n' = (3n+1)/2 \ /2$

#### $\frac{1}{12}$ of all integers are in the lattice under this rule?

Are these all "$\textit{ \(2n+1\)  easy}$"? .. would give us 1/6 of all integers.


In [3]:
"""
Code from previous notebooks,

TODO: Remove unneed functions not needed for mrTup representation

"""

def T(n):
    """ Compute next value in simplified Collatz sequence.
    """
    if n & 1 == 0:
        return n//2
    else:
        return (3*n + 1)//2
#
def L_T(n):
    """ Compute binary label-string for a given Collatz number
    """
    if n == 1:
        return "1"

    S = ""
    while n != 1:
        if n & 1 == 0:
            n = n//2
            S = S + "1"
        else:
            n = (3*n + 1)//2
            S = S + "0"
    return S
#
def Ay_L(L):
    """ Generate A matrix and y vector from label string
    """
    rank = len(L) + 2    
    A = np.zeros((rank,rank))
    y = np.zeros((rank))
    for row in range(rank-3):
        if L[row-2] == "0":
            a_val = -1.0
            y_val = 0.0
        else:
            a_val = -3.0
            y_val = 1.0
        A[row][row] = a_val
        A[row][row+1] = 2.0
        y[row] = y_val
    #
    # Last 3 rows are always the same
    row = rank - 3
    A[row][row] = -1
    A[row][row+1] = 2
    y[row] = 0
    row = rank - 2
    A[row][row] = -3
    A[row][row+1] = 2
    y[row] = 1
    row = rank - 1
    A[row][row] = 1
    A[row][row-2] = -1
    y[row] = 0
    
    return A, y
#
def solve_Ay_L(L):
    """ Solve for the x vector given the label-string
    """
    A, y = Ay_L(L)
    return A, np.linalg.solve(A, y), y
#
def x0_L(L):
    """ Get the x[0] value given a label-string
    """
    A, x, y = solve_Ay_L(L)
    return round(x[0])  # clean up mantisa garbage
#

def countZeros(label):
    zero_count = 0
    for bit in label:
        if bit == "0":
            zero_count += 1
    return zero_count
#

def Z(L):
    """ Indexes of zeros in label string
    """
    for i in range(len(L)):
        if L[i] == "0":
            yield i
#
def a_b_c_L(L):
    """ Get the (power-of-two, power-of-three, zero-sum-accumulator) tuple for a node given its label
    """
    a = len(L)
    b = 0
    for bit in L:
        if bit == "0":
            b += 1
    ZZ = [(j,i) for j, i in enumerate(Z(L))]
    c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in ZZ)
    S = [zz[1] for zz in ZZ]
    return (a,b,c,S)
#
def val_a_b_c(a_b_c):
    """ Get the value for a node given the tuple (power-of-two, power-of-three, zero-sum-accumulator)
    """
    a, b, c = a_b_c
    f = Fraction( ((2**a) - c), (3**b) )
    return (f.numerator, f.denominator)
#
def val_a_b_c_L(a_b_c_L):
    a_b_c = a_b_c_L[0:3]
    return val_a_b_c(a_b_c)
#
def val_L(L):
    """ Get the value for a node given the label string
    """
    return val_a_b_c_L(a_b_c_L(L))
#
def collatzPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#

def collatzPath2(n_d_tup, truncate_at=100):
    """
    Collatz Path for rationals with loop detection and "too long" cut-off
    """
    chain = [n_d_tup]
    path = []
    governor = truncate_at
    while n_d_tup != (1, 1):
        if (n_d_tup[0] & 1) == 0:
            n_d_tup = (n_d_tup[0]//2, n_d_tup[1])
            path.append("1")
        else:
            f = Fraction((3 * n_d_tup[0] + n_d_tup[1])//2, n_d_tup[1])
            n_d_tup = (f.numerator, f.denominator)
            path.append("0")
        if n_d_tup in chain:
            path.insert(0,"↺")
            break
        
        governor -= 1
        if governor == 0:
            path.insert(0,"∀")
            break
        #
        chain.append(n_d_tup)
    #
    return ("".join(path), chain)
#

    
#
#  Mixed Radix form and functions
#

N_1 = ((0,0), [])

def mr_TupItemValue(a_b, a_0):
    a,b = a_b
    val = (2**a)*(3**(a_0 + b))
    # print(f"val {a_b}*(3**{a_0}) = {val}")
    return val
#
def mrTupValue(mr_tup):
    # Multiplying the numerator by 3 ** the generation keeps us in integer land
    a_0 = mr_tup[0][0]
    total = mr_TupItemValue(mr_tup[0], a_0)
    for a_b in mr_tup[1]:
        total -= mr_TupItemValue(a_b, a_0)
    frac = Fraction(total, 3**a_0)
    return (frac.numerator, frac.denominator)
#

def F_0(mr_tup):
    return ( (mr_tup[0][0]+1, mr_tup[0][1]-1), mr_tup[1] + [(mr_tup[0][0], -(len(mr_tup[1])+1))] )
#

def F_1(mr_tup):
    return ((mr_tup[0][0]+1, mr_tup[0][1]), mr_tup[1])
#
def mrTupFromPath(label):
    mr_tup = N_1
    for bit in label:
        if bit == "1":
            mr_tup = F_1(mr_tup)
        else:
            mr_tup = F_0(mr_tup)
    return mr_tup
#
def mrTupToPath(T):
    """
    Convert mrTup to node label
    The T[1] list encodes the positions of the zeros in the numerator power of two values
    """
    S = ["1"] * T[0][0]
    for j in range(len(T[1])):
        S[(T[1][j][0])] = "0" 
    return "".join(S)
#

def mrTupFromValue(n):
    label = collatzPath(n)
    return mrTupFromPath(label)
#

def strip_01(label):
    while len(label) > 2 and label[-2:] == "01":
        label = label[0:-2]
    return label
#

LABEL_RX = re.compile('^(?P<prefix>.*?)((?P<inttag>111)(?P<tail>((01)*)))$')

def split_int_label(s):
    """
    Splits label into 3 parts and returns prefix and suffix if matches integer-candidate pattern
    """
    match = LABEL_RX.search(s)
    if match:
        return (match.group('prefix'), match.group('tail'))
    else:
        return None  # or raise an error if preferred
#

def generationLabels(a):
    if a == 0:
        return ""
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        yield label
#
def generationTups(a):
    for label in generationLabels(a):
        mrTup = mrTupFromPath(label)
        yield (label, mrTup, mrTupValue(mrTup))
    
def generationIntCandidateLabels(a):
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        head_tail = split_int_label(label)
        if head_tail:
            # return int candidate with stripped tail
            yield head_tail[0] + "111"
#

def mrIntTupsForGeneration(aa):
    for label in generationIntCandidateLabels(aa):
        mrTup = mrTupFromPath(label)
        yield (label, mrTup, mrTupValue(mrTup))
#
def generationGenNums(a):
    vals = []
    bb = 3**(a)
    for infoTup in generationTups(a):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] * (bb//val_tup[1]))
    vals.sort(reverse=True)
    for idx, val in enumerate(vals):
        if idx < len(vals) - 1:
            delta = val - vals[idx+1]
        else:
            delta = None
        print(f'{val}\t{delta}')
#        
def generationPairGenNums(a):
    vals = []
    bb = 3**(a+1)
    for infoTup in generationTups(a):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] * (bb//val_tup[1]))
    for infoTup in generationTups(a+1):
        _, __, val_tup = infoTup
        vals.append(val_tup[0] *  (bb//val_tup[1]))
    vals.sort(reverse=True)
    for idx, val in enumerate(vals):
        if idx < len(vals) - 1:
            delta = val - vals[idx+1]
        else:
            delta = None
        print(f'{val}\t{delta}')
#
def mr2Nplus_1(T):
    B = len(T[1])  
    L = [(0, -1)]

    # Keep initial zeros
    idx = 0
    for idx, val in enumerate(T[1]):
        if T[1][idx][0] != idx:
            break
        L.append( (T[1][idx][0] + 1, T[1][idx][1]-1) )
    # Remove the first tuple where (a, -a) is true
    match = False
    for i in range(idx, B, 1):
        if (not match) and (T[1][i][0] == -T[1][i][1]):
            match = True
        else:
            L.append( (T[1][i][0]+1, T[1][i][1]) )
    if not match:
        return None
    return ( (T[0][0] + 1, T[0][1]), L)
#

'''
Do not think this works yet:
def mr2Nplus_1_inv(T):
    """
    Find the inverse tuple of mr2Nplus_1

    We know the zero was removed when L item index matched numerator and denominator
    """
    B = len(T[1])  
    L = []

    # Keep initial zeros after poping (0, -1) off the front
    idx = 0
    for idx, val in enumerate(T[1]):
        if idx == 0:
            if val == (0, -1):
                continue
            else:
                # This tuple was not generated by mr2Nplus_1
                return None
        else:
            L.append( (T[1][idx][0] - 1, T[1][idx][1] + 1) )
    inserted = False
    for i in range(idx, B, 1):
        if (not inserted) and i == (len(L)+1) and (T[0][i][0] > i):
            # Next zero in 2n+1 L list is larger than i ... insert the (i, -i) term
            L.append( (i, -i) )    
            inserted == True
        L.append( (T[1][i][0]-1, T[1][i][1]) )
        
    return ( (T[0][0] - 1, T[0][1]), L)
#
'''

def lattice2N_plus1_pairs(a):
    """
    For a given depth in the tree, generate all 2n+1 pairs in the tree
    """
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        label = strip_01(label)
        val = mrTupValue(mrTupFromPath(label))
        f = Fraction(2 * val[0] + 1, val[1])
        val_ = (f.numerator, f.denominator)
        label_, chain_ = collatzPath2(val_)
        if (len(label_) == 0):
            d = len(label)
        elif label_[0] in ["↺", "∀"]:
            d = 100 + len(label)
        else:
            d = distance(label, label_)
        yield (len(label), d, (val, label), (val_, label_))
#



In [4]:
def checkTupEasy(label):
    T = mrTupFromPath(label)
    TT = mr2Nplus_1(T)
    if TT is not None:
        return True
    else:
        return False
#

In [5]:
# No label = "11+" lattice nodes are "easy" and only 2 -> 5 is somewhat neighborly
for i in range(0, 20):
    val = 2*(2**i) + 1
    tup_2n_plus1 =  mrTupFromValue(val)    
    print ((i, val, mrTupToPath(tup_2n_plus1), tup_2n_plus1))

(0, 3, '00111', ((5, -2), [(0, -1), (1, -2)]))
(1, 5, '0111', ((4, -1), [(0, -1)]))
(2, 9, '0100010110111', ((13, -6), [(0, -1), (2, -2), (3, -3), (4, -4), (6, -5), (9, -6)]))
(3, 17, '010110111', ((9, -3), [(0, -1), (2, -2), (5, -3)]))
(4, 33, '010100110010110111', ((18, -8), [(0, -1), (2, -2), (4, -3), (5, -4), (8, -5), (9, -6), (11, -7), (14, -8)]))
(5, 65, '0101011100010110111', ((19, -8), [(0, -1), (2, -2), (4, -3), (8, -4), (9, -5), (10, -6), (12, -7), (15, -8)]))
(6, 129, '01010100011000010100100010000101100010010000001100001110101011101100011110111', ((77, -44), [(0, -1), (2, -2), (4, -3), (6, -4), (7, -5), (8, -6), (11, -7), (12, -8), (13, -9), (14, -10), (16, -11), (18, -12), (19, -13), (21, -14), (22, -15), (23, -16), (25, -17), (26, -18), (27, -19), (28, -20), (30, -21), (33, -22), (34, -23), (35, -24), (37, -25), (38, -26), (40, -27), (41, -28), (42, -29), (43, -30), (44, -31), (45, -32), (48, -33), (49, -34), (50, -35), (51, -36), (55, -37), (57, -38), (59, -39), (63, -40

In [6]:
mrTupFromPath("11111"), mrTupValue(mrTupFromPath("11111"))



(((5, 0), []), (32, 1))

In [7]:
PARTS_RX_tail = re.compile('^(?P<head>[01]*?)((?P<inttag>(111)?)(?P<tail>((01)*)))$')
PARTS_RX_head = re.compile('^(?P<prefix>0*)(?P<ones>1*)(?P<thezero>0?)(?P<remainder>[01]*)$')
RULE1_RX = re.compile('^(?P<prefix>0*)(?P<ones>1+)(?P<thezero>0)(?P<remainder>[01]*)((?P<inttag>111)(?P<tail>((01)*)))$')

In [8]:
def mrTupLabelParts(label):
    match_tail = PARTS_RX_tail.search(label)
    if match_tail:
        match_head = PARTS_RX_head.search(match_tail.group('head'))
        if match_head:
            P = (match_head.group('prefix'), match_head.group('ones'), match_head.group('thezero'), match_head.group('remainder'), match_tail.group('inttag'), match_tail.group('tail'))
    else:
        P = ("","","",label, "", "")
    return P
#

In [9]:
def genEasy2n_plus_1(a):
    """
    Generate all "easy" 2n+1 mrTups
    """
    if a <= 5:
        yield None
    else:
        for label in generationLabels(a):
            T = mrTupFromPath(label)
            val = mrTupValue(T)
            if val[1] == 1:
                did_match = False
                match = RULE1_RX.search(label)
                if match:
                    if a <= 12:
                        print((match.group('prefix'), match.group('ones'), match.group('thezero'), match.group('remainder'), match.group('inttag'), match.group('tail')))
                    did_match = True
                    T_ = mr2Nplus_1(T)
                    if T_ is None:
                        print("EASY NOT EASY:")
                        print((val[0], label))
                    else:
                        val_ = mrTupValue(T_)
                        if  val_[1] != 1 or val_[0] != (2*val[0] + 1):
                            print("EASY NOT EASY -- check fail:")
                            print((val[0], label))
                yield((label, val, did_match))
#

In [10]:
list (genEasy2n_plus_1(7))

('', '111', '0', '', '111', '')
EASY NOT EASY:
(40, '1110111')
('', '11', '0', '0', '111', '')
EASY NOT EASY:
(12, '1100111')
('', '1', '0', '11', '111', '')
('', '1', '0', '', '111', '01')
('0', '11', '0', '', '111', '')
EASY NOT EASY:
(13, '0110111')


[('1111111', (128, 1), False),
 ('1111101', (32, 1), False),
 ('1110111', (40, 1), True),
 ('1110101', (8, 1), False),
 ('1100111', (12, 1), True),
 ('1011111', (42, 1), True),
 ('1011101', (10, 1), True),
 ('1010101', (2, 1), False),
 ('0110111', (13, 1), True),
 ('0011101', (3, 1), False)]

In [11]:
mrTupValue(mrTupFromPath('01111111'))

(85, 1)

In [12]:
mrTupFromPath('01111111')

((8, -1), [(0, -1)])

In [13]:
mrTupFromValue(42)

((7, -1), [(1, -1)])

In [14]:
collatzPath(42)

'1011111'

### The genEasy2n_plus_1 is not unique

Can be applied to all three of these and give the same final tuple label '01111111' (85)

-- Clearly only one can be the correct $n$ of $n' = 2n+1$

- 1011111 (42)
- 1101111 (124/3)
- 1110111 (40)

What cases work with only left-most possible zero position and which work with other zero positions?

In [15]:
# Correct n = 42
mrTupValue(mrTupFromPath('1011111'))

(42, 1)

In [16]:
# Incorrect
mrTupValue(mrTupFromPath('1101111'))

(124, 3)

In [17]:
# Incorrect
mrTupValue(mrTupFromPath('1110111'))

(40, 1)

### 13 case

In [18]:
mrTupValue(mrTupFromPath('0110111'))

(13, 1)

In [19]:
mrTupValue(mrTupFromPath('00111111'))

(251, 9)

In [20]:
mrTupToPath(mrTupFromValue(2*13+1))

'0010000010100100010000101100010010000001100001110101011101100011110111'

### 81 case

In [21]:
mrTupToPath(mrTupFromValue(81))

'0101100011110111'

In [22]:
mrTupValue(mrTupFromPath('01110111'))

(79, 3)

In [23]:
list (genEasy2n_plus_1(8))

('', '1111', '0', '', '111', '')
EASY NOT EASY:
(80, '11110111')
('', '111', '0', '0', '111', '')
EASY NOT EASY:
(24, '11100111')
('', '11', '0', '11', '111', '')
EASY NOT EASY:
(84, '11011111')
('', '11', '0', '', '111', '01')
EASY NOT EASY:
(20, '11011101')
('', '1', '0', '110', '111', '')
('', '1', '0', '0', '111', '01')


[('11111111', (256, 1), False),
 ('11111101', (64, 1), False),
 ('11110111', (80, 1), True),
 ('11110101', (16, 1), False),
 ('11100111', (24, 1), True),
 ('11011111', (84, 1), True),
 ('11011101', (20, 1), True),
 ('11010101', (4, 1), False),
 ('10110111', (26, 1), True),
 ('10011101', (6, 1), True),
 ('01111111', (85, 1), False),
 ('01111101', (21, 1), False),
 ('01110101', (5, 1), False),
 ('01010101', (1, 1), False)]

In [24]:
list(genEasy2n_plus_1(12))

('', '11111111', '0', '', '111', '')
EASY NOT EASY:
(1280, '111111110111')
('', '1111111', '0', '0', '111', '')
EASY NOT EASY:
(384, '111111100111')
('', '111111', '0', '11', '111', '')
EASY NOT EASY:
(1344, '111111011111')
('', '111111', '0', '', '111', '01')
EASY NOT EASY:
(320, '111111011101')
('', '11111', '0', '110', '111', '')
EASY NOT EASY:
(416, '111110110111')
('', '11111', '0', '0', '111', '01')
EASY NOT EASY:
(96, '111110011101')
('', '1111', '0', '1111', '111', '')
EASY NOT EASY:
(1360, '111101111111')
('', '1111', '0', '11', '111', '01')
EASY NOT EASY:
(336, '111101111101')
('', '1111', '0', '', '111', '0101')
EASY NOT EASY:
(80, '111101110101')
('', '111', '0', '11110', '111', '')
EASY NOT EASY:
(424, '111011110111')
('', '111', '0', '110', '111', '01')
EASY NOT EASY:
(104, '111011011101')
('', '111', '0', '10110', '111', '')
EASY NOT EASY:
(136, '111010110111')
('', '111', '0', '0', '111', '0101')
EASY NOT EASY:
(24, '111001110101')
('', '11', '0', '111111', '111', '')
E

[('111111111111', (4096, 1), False),
 ('111111111101', (1024, 1), False),
 ('111111110111', (1280, 1), True),
 ('111111110101', (256, 1), False),
 ('111111100111', (384, 1), True),
 ('111111011111', (1344, 1), True),
 ('111111011101', (320, 1), True),
 ('111111010101', (64, 1), False),
 ('111110110111', (416, 1), True),
 ('111110011101', (96, 1), True),
 ('111101111111', (1360, 1), True),
 ('111101111101', (336, 1), True),
 ('111101110101', (80, 1), True),
 ('111101010101', (16, 1), False),
 ('111011110111', (424, 1), True),
 ('111011011101', (104, 1), True),
 ('111010110111', (136, 1), True),
 ('111001110101', (24, 1), True),
 ('110111111111', (1364, 1), True),
 ('110111111101', (340, 1), True),
 ('110111110101', (84, 1), True),
 ('110111010101', (20, 1), True),
 ('110101111111', (452, 1), True),
 ('110101010101', (4, 1), False),
 ('110011110111', (140, 1), True),
 ('110010110111', (44, 1), True),
 ('101111110111', (426, 1), True),
 ('101111011101', (106, 1), True),
 ('101110110111', 

In [25]:
ints_gen_20 = list(genEasy2n_plus_1(20))

EASY NOT EASY:
(327680, '11111111111111110111')
EASY NOT EASY:
(98304, '11111111111111100111')
EASY NOT EASY:
(344064, '11111111111111011111')
EASY NOT EASY:
(81920, '11111111111111011101')
EASY NOT EASY:
(106496, '11111111111110110111')
EASY NOT EASY:
(24576, '11111111111110011101')
EASY NOT EASY:
(348160, '11111111111101111111')
EASY NOT EASY:
(86016, '11111111111101111101')
EASY NOT EASY:
(20480, '11111111111101110101')
EASY NOT EASY:
(108544, '11111111111011110111')
EASY NOT EASY:
(26624, '11111111111011011101')
EASY NOT EASY:
(34816, '11111111111010110111')
EASY NOT EASY:
(6144, '11111111111001110101')
EASY NOT EASY:
(349184, '11111111110111111111')
EASY NOT EASY:
(87040, '11111111110111111101')
EASY NOT EASY:
(21504, '11111111110111110101')
EASY NOT EASY:
(5120, '11111111110111010101')
EASY NOT EASY:
(115712, '11111111110101111111')
EASY NOT EASY:
(35840, '11111111110011110111')
EASY NOT EASY:
(11264, '11111111110010110111')
EASY NOT EASY:
(109056, '11111111101111110111')
EASY NO

In [26]:
easy_count = 0
not_easy_count = 0
for tup in ints_gen_20:
    if tup[2]:
        easy_count += 1
    else:
        not_easy_count += 1
#
easy_count, not_easy_count

(467, 21)

# What does "near" look like in generation 20?


In [27]:
ints_gen_20_byval = sorted([ (T[1][0], T[0]) for T in ints_gen_20])
ints_gen_20_byval

[(1, '01010101010101010101'),
 (4, '11010101010101010101'),
 (5, '01110101010101010101'),
 (6, '10011101010101010101'),
 (11, '00101101110101010101'),
 (14, '10001011011101010101'),
 (15, '00001111011101010101'),
 (16, '11110101010101010101'),
 (18, '10100010110111010101'),
 (19, '00110010110111010101'),
 (20, '11011101010101010101'),
 (21, '01111101010101010101'),
 (24, '11100111010101010101'),
 (25, '01001100101101110101'),
 (26, '10110111010101010101'),
 (33, '01010011001011011101'),
 (34, '10101101110101010101'),
 (35, '00111101110101010101'),
 (43, '00101011100010110111'),
 (44, '11001011011101010101'),
 (45, '01101011011101010101'),
 (46, '10001111011101010101'),
 (56, '11100010110111010101'),
 (58, '10110010110111010101'),
 (60, '11000011110111010101'),
 (61, '01100011110111010101'),
 (64, '11111101010101010101'),
 (72, '11101000101101110101'),
 (74, '10111000101101110101'),
 (76, '11001100101101110101'),
 (77, '01101100101101110101'),
 (80, '11110111010101010101'),
 (81, '01011

In [29]:
mrTupValue(mrTupFromPath('01111111110101111111'))

(115711, 3)

In [30]:
115711/3

38570.333333333336

In [31]:
mrTupFromPath('01111111110101111111')

((20, -3), [(0, -1), (10, -2), (12, -3)])

# Interesting "Near values in generation 20"

$\Large{\frac{2^{20}}{3^3} - \frac{2^{12}}{3^3} - \frac{2^{10}}{3^2} - ... }$

$\Large{38570\frac{2}{3} - c_0 \in [\frac{1}{3}, 2\frac{2}{3}, 8\frac{2}{3}, 32\frac{2}{3}, 128\frac{2}{3}] }$

```
       (38400, '11111111100101111111'),
         Δ128          x x
       (38528, '11111110110101111111'),
          Δ32        x x
       (38560, '11111011110101111111'),
           Δ8      x x
       (38568, '11101111110101111111'),
           Δ2    x x
       (38570, '10111111110101111111'),
        Δ 1/3   xx
 (38570 + 1/3, '01111111110101111111')
```

But we have to go out to generation 55 to pick up 32589 which is between 38568 and 38570
and out to generation 109 to pick up 38571.
```
       (38568, '11101111110101111111'),
        32589  '0100000000111101010101100111100010110110111100011110111'
       (38570, '10111111110101111111'),
        38571  '0010101010011101100101100001010110111011010000010100100010000101100010010000001100001110101011101100011110111'
```



In [38]:
collatzPath(38571)

'0010101010011101100101100001010110111011010000010100100010000101100010010000001100001110101011101100011110111'

In [39]:
len('0010101010011101100101100001010110111011010000010100100010000101100010010000001100001110101011101100011110111')

109

In [35]:
((2**20) - (2**12) - 3*(2**10))/27

38570.666666666664

In [34]:
(0.444444444445) * 27

12.000000000015

In [40]:
# 10111111110101111111 * 11101111110101111111 => Much longer path with much more than 24 zeros.
collatzPath(38570 * 38568)

'111101001011101000100101011001010111000110100101001110110010010000100110111001110010010100000001001011011010101011011110000011100100010000101100010010000001100001110101011101100011110111'

In [42]:
countZeros(collatzPath(38570 * 38568))

98

In [135]:
mrTupFromValue(20), mrTupFromValue(21)

(((6, -1), [(2, -1)]), ((6, -1), [(0, -1)]))

In [136]:
mrTupFromPath('00110010110111010101'), mrTupFromPath('11011101010101010101')

(((20, -9),
  [(0, -1),
   (1, -2),
   (4, -3),
   (5, -4),
   (7, -5),
   (10, -6),
   (14, -7),
   (16, -8),
   (18, -9)]),
 ((20, -8),
  [(2, -1),
   (6, -2),
   (8, -3),
   (10, -4),
   (12, -5),
   (14, -6),
   (16, -7),
   (18, -8)]))

In [137]:
mrTupFromValue(3*19)

((22, -10),
 [(0, -1),
  (2, -2),
  (3, -3),
  (5, -4),
  (7, -5),
  (11, -6),
  (12, -7),
  (13, -8),
  (15, -9),
  (18, -10)])

In [138]:
collatzPath(3*19)

'0100101011100010110111'

In [139]:
mrTupValue(mrTupFromPath('00110010110111'))

(271, 3)

In [140]:
mrTupFromValue(9)

((13, -6), [(0, -1), (2, -2), (3, -3), (4, -4), (6, -5), (9, -6)])

In [141]:
collatzPath(9)

'0100010110111'

In [142]:
mrTupFromValue(28)

((13, -5), [(2, -1), (3, -2), (4, -3), (6, -4), (9, -5)])

In [143]:
collatzPath(28)

'1100010110111'

In [147]:
mrTupValue(mrTupFromPath('01011100010110111'))

(49, 1)

In [ ]:
#[(tup[0], 4*tup[0] - 1, mrTupValue(mrTupFromPath('01'+tup[1]))) for tup in ints_gen_20_byval]

In [161]:
mrTupFromValue(19)

((14, -6), [(0, -1), (1, -2), (4, -3), (5, -4), (7, -5), (10, -6)])

In [162]:
collatzPath(19)

'00110010110111'

In [156]:
#19 evaluation
b = 9
for c_d in [(0, -1), (1, -2), (4, -3),   (5, -4), (7, -5),   (10, -6),             (14, -7), (16, -8), (18, -9)]:
    d_ = c_d[1] + b
    b_val = 2**(c_d[0])*3**(d_)
    print(b_val)


6561
4374
11664
7776
10368
27648
147456
196608
262144


In [157]:
# last 3 are big decrementers and parallel between 19 and 20 (with a 3X factor that matches b):
2**(20) - (147456 + 196608 + 262144)

442368

In [159]:
442368 // 3

147456

In [160]:
# 20 evaluation
b = 8
for c_d in [(2, -1),   (6, -2), (8, -3),   (10, -4),   (12, -5), (14, -6), (16, -7), (18, -8)]:
    d_ = c_d[1] + b
    b_val = 2**(c_d[0])*3**(d_)
    print(b_val)


8748
46656
62208
82944
110592
147456
196608
262144


In [ ]:
# Surpise:  3*(14, -6) = sum([(14, -7), (16, -8), (18, -9)])

In [163]:
2**(20) - (147456 + 196608 + 262144)

442368

In [164]:
(2**20)/(3**9)

53.273179901437786

In [170]:
0.273179901437786 * (3**9)

5376.999999999942

In [171]:
f = Fraction(5377, 3**9)
f.numerator, f.denominator

(5377, 19683)

In [172]:
3**9

19683

In [173]:
# (2**20)/(3**9) -> 53 5377/19683  - 19 = 34 5377/19683

In [ ]:
def mr2Nplus_1(T):
    B = len(T[1])  
    L = [(0, -1)]

    # Keep initial zeros
    idx = 0
    for idx, val in enumerate(T[1]):
        if T[1][idx][0] != idx:
            break
        L.append( (T[1][idx][0] + 1, T[1][idx][1]-1) )
    # Remove the first tuple where (a, -a) is true
    match = False
    for i in range(idx, B, 1):
        if (not match) and (T[1][i][0] == -T[1][i][1]):
            match = True
        else:
            L.append( (T[1][i][0]+1, T[1][i][1]) )
    if not match:
        return None
    return ( (T[0][0] + 1, T[0][1]), L)
#


In [217]:
def splitOutInitialZeros(L):
    L_a = []
    L_z = []
    initial = True
    for i in range(len(L)):
        if initial and L[i][0] == i:
            L_a.append(L[i])
        else:
            initial = False
            L_z.append(L[i])
    return L_a, L_z
#


In [220]:
T_27 = mrTupFromValue(27)
print(splitOutInitialZeros(T_27[1]))

([(0, -1), (1, -2)], [(3, -3), (4, -4), (5, -5), (6, -6), (7, -7), (9, -8), (11, -9), (12, -10), (14, -11), (15, -12), (16, -13), (18, -14), (19, -15), (20, -16), (21, -17), (23, -18), (26, -19), (27, -20), (28, -21), (30, -22), (31, -23), (33, -24), (34, -25), (35, -26), (36, -27), (37, -28), (38, -29), (41, -30), (42, -31), (43, -32), (44, -33), (48, -34), (50, -35), (52, -36), (56, -37), (59, -38), (60, -39), (61, -40), (66, -41)])


In [248]:

def mrTup2NIdentityElement(T):
    """ 
    returns the index of the L list item. if any, that allows
    a 2n operation to be transformed into a 2n+1 operation by
    the removal of a single element.

    Brute force for now
    """
    val = mrTupValue(T)
    f = Fraction(2*val[0] + val[1], val[1])
    val_goal = (f.numerator, f.denominator)

    a = T[0][0]
    b = T[0][1]
    L = T[1]

    L_head, L_tail = splitOutInitialZeros(L)

    L_head_ = [(0, -1)]
    for c_d in L_head:
        L_head_.append((c_d[0] + 1, c_d[1] -1))
    
    for i in range(len(L_tail)):
        # which one of these, when we ditch it gives us our goal value?
        L_tail_ = [(c_d[0] + 1, c_d[1]) for idx, c_d in enumerate(L_tail) if idx != i]
        T_ = ((a+1, b), L_head_ + L_tail_)
        # print(T_)
        val_ = mrTupValue(T_)
        if (val_ == val_goal):
            # i seems to always be zero
            return (i, i + len(L_head_))
    return None
#

In [249]:
mrTup2NIdentityElement(T_27)

(0, 3)

In [250]:
list(mrIntTupsForGeneration(4))

[('1111', ((4, 0), []), (16, 1)), ('0111', ((4, -1), [(0, -1)]), (5, 1))]

In [268]:
for PTV in generationTups(7):
    label, T, val = PTV
    easy_idx = mrTup2NIdentityElement(T)
    if easy_idx is not None:
        print((easy_idx, mrTupLabelParts(mrTupToPath(T)), mrTupValue(T)))
    else:
        print((mrTupLabelParts(mrTupToPath(T))))


('', '1111', '', '', '111', '')
('', '111111', '0', '', '', '')
('', '11', '', '', '111', '01')
('', '11111', '0', '0', '', '')
('', '1111', '0', '11', '', '')
('', '1111', '0', '10', '', '')
('', '1111', '0', '', '', '01')
('', '1111', '0', '00', '', '')
('', '111', '0', '', '111', '')
('', '111', '0', '110', '', '')
('', '', '', '', '111', '0101')
('', '111', '0', '100', '', '')
('', '111', '0', '011', '', '')
('', '111', '0', '010', '', '')
('', '111', '0', '0', '', '01')
('', '111', '0', '000', '', '')
('', '11', '0', '1', '111', '')
('', '11', '0', '1110', '', '')
('', '11', '0', '11', '', '01')
('', '11', '0', '1100', '', '')
('', '11', '0', '1011', '', '')
('', '11', '0', '1010', '', '')
('', '11', '0', '10', '', '01')
('', '11', '0', '1000', '', '')
('', '11', '0', '0', '111', '')
('', '11', '0', '0110', '', '')
('', '11', '0', '', '', '0101')
('', '11', '0', '0100', '', '')
('', '11', '0', '0011', '', '')
('', '11', '0', '0010', '', '')
('', '11', '0', '00', '', '01')
('', '11

In [215]:
T27_F0 = mrTupValue(F_0(mrTupFromValue(27)))
(T27_F0, T27_F0[0]/T27_F0[1])

((593129465116011091795, 109418989131512359209), 5.420717828083019)

In [ ]:
F_0(mrTupFromValue(27))


In [270]:
collatzPath(27), collatzPath(54)

('0010000010100100010000101100010010000001100001110101011101100011110111',
 '10010000010100100010000101100010010000001100001110101011101100011110111')